# Pathweigh Analysis

This notebook demonstrates pathway analysis of RNA-seq sequencing data using PathWeigh. It is recommended to clone this repository by using:
```
git clone https://github.com/zurkin1/Yada.git
!pip install -r ../requirements.txt
```
and then run it in a Jupyter notebook.

The input file `./data/input.csv` is located in the data folder.

In [1]:
from udp import *

# We need can configure the following parameter is_rnaseq == True in order to select RNA-seq data
# processing or microarray data processing. 
calc_udp_multi_process(is_rnaseq = True)

Fri Sep  6 10:17:41 2024 Calculate UDP, is_rnaseq: True
......................................

## Calculate Activity For all Samples and For All Pathways.

In [3]:
from notebooks.udp import *
from activity import *


udp = pd.read_csv('./data/output_udp.csv', index_col=0, encoding="utf-8")
activity_obj = path_activity(udp, is_rnaseq = True)
activity = activity_obj.calc_activity_consistency_multi_process() #Output is saved to output_activity.csv.

activity_obj.activity.head()

Tue Jul 18 09:12:10 2023 Init activity object
Tue Jul 18 09:12:11 2023 Calculate activity and consistency...
.Tue Jul 18 09:12:24 2023 Done.


sampleID,17-002,17-006,17-019,17-023,17-026,17-027,17-030,17-034,17-040,17-041,17-042,17-043,17-045,17-047,17-054,17-055,17-057,17-058,17-060,17-061
path_name,,,,,,,,,,,,,,,,,,,,
1- and 2-Methylnaphthalene degradation(Kegg),0.647734,0.656807,0.557137,0.424184,0.473169,0.438492,0.478672,0.6425,0.63545,0.594516,0.56237,0.632876,0.480193,0.594516,0.53638,0.595516,0.558011,0.575818,0.64113,0.576818
1-4-Dichlorobenzene degradation(Kegg),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3-Chloroacrylic acid degradation(Kegg),0.798822,0.817806,0.796039,0.7341,0.68915,0.723182,0.845878,0.791842,0.741356,0.741112,0.827948,0.796039,0.755424,0.741112,0.675395,0.759734,0.678203,0.835736,0.685988,0.75756
Acute myeloid leukemia(Kegg),0.359899,0.365481,0.398714,0.345536,0.344096,0.316362,0.377824,0.383984,0.460113,0.342276,0.392314,0.2915,0.337403,0.355758,0.366264,0.423215,0.361883,0.257353,0.37749,0.33395
Adherens junction(Kegg),0.233236,0.243647,0.28955,0.228519,0.178912,0.215123,0.280066,0.306218,0.269894,0.281987,0.309846,0.306717,0.294797,0.255783,0.27215,0.302786,0.247442,0.19818,0.21534,0.297161


## Draw a Heatmap For Path Activity.

The following UI demonstrations were done using the following libraries versions:

- plotly 5.9.0
- ipywidgets 7.6.5 (plotly currently cannot work with ipywidgets version 8). It might be possible to run other combinations.

In [4]:
import plotly.express as px
import plotly.graph_objects as go


sample_num = 0
path_id = 708708 #1-and 2-Methylnaphthalene degradation(Kegg).

df = activity_obj.activity.transpose()

#for col in df:
#    if df[col].mean() < 0.5:
#        df.drop(col, inplace=True, axis=1)

#fig = px.imshow(df, text_auto = True, aspect = 'auto', width = 1000, height = 800)
fig = go.FigureWidget(go.Heatmap(x=df.columns, y=df.index, z=df.to_numpy()))
fig.update_layout(width = 1000, height = 1000)

#Create our callback function.
hmap = fig.data[0]
fig.layout.hovermode = 'closest'

def update_point(trace, points, selector):
    path_id = points.xs[0]
    sample_num = points.ys[0]
    activity_obj.graphparser(path_id=path_id, sample_num=sample_num)
    #xml_result = activity_obj.xmlparser(path_id=path_id, sample_num=sample_num)


hmap.on_click(update_point)

fig

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'cab899cc-387e-4f91-95cb-9e0331920c2f',
 …

## Download a KGML File to Import to Cytoscape.


- Configure sample_num.
- Configure path_id for analysis and export. Use the following table:
https://github.com/zurkin1/Pathweigh/blob/master/README.md.

In [5]:
import bs4


sample_num = 0
path_id = 708708

xml_result = activity_obj.xmlparser(path_id=path_id, sample_num=sample_num)
print(bs4.BeautifulSoup(xml_result).prettify())

Create Kegg XML for path: 708708, sample: 0
<?xml version="1.0"?>
<!DOCTYPE pathway SYSTEM "https://www.kegg.jp/kegg/xml/KGML_v0.7.2_.dtd">
<html>
 <body>
  <pathway name="path:hsa708708" number="111" org="hsa" title="1- and 2-Methylnaphthalene degradation(Kegg)">
   <entry id="708648" link="https://www.kegg.jp/dbget-bin/www_bget?hsa:4967+hsa:708648" name="hsa:4967 hsa:fm_708637" type="protein">
    <graphics bgcolor="#42ECEF" fgcolor="#FF4400" height="17" name="fm_708637" type="rectangle" width="60" x="0.8885235533793071">
    </graphics>
   </entry>
   <entry id="708709" link="https://www.kegg.jp/dbget-bin/www_bget?C00158" name="708709" type="reaction">
    <graphics bgcolor="#FFF8F8" fgcolor="#FF4400" height="15" name="reaction" type="circle" width="60" x="0.8885235533793071" y="0.8885235533793071">
    </graphics>
   </entry>
   <relation entry1="708648" entry2="708709" type="PPrel">
    <subtype name="activation" value="--&gt;">
    </subtype>
   </relation>
   <entry id="708651" 

C:\Users\danili\AppData\Local\anaconda3\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning:

It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.



## Draw a Pathway Activity Map.

We can select a specific sample and a specific pathway and draw the activity map.

In [6]:
sample_num = 0
path_id = 708708

activity_obj.graphparser(path_id, sample_num)

## Troubleshooting
- It was reported on problems running in Pycharm due to multiprocessing issues. There are no problems running on the command line or in Colab.